In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_mps = pd.read_excel('MPS.xlsx')

In [2]:
# Tent's Gross Requirements across 12 weeks
tent_gross_requirements = [0, 100, 100, 0, 100, 0, 100, 100, 100, 0, 100, 100]

# Part quantities needed per tent
part_quantities = {
    1000: 1, 1100: 1, 1110: 9, 1120: 4, 1130: 5,
    1131: 10, 1132: 9, 1133: 10, 1200: 1, 1210: 1,
    1211: 1, 1212: 6.5, 1220: 1, 1221: 1, 1222: 1,
    999: 4.5, 1230: 3, 1240: 24, 1250: 8, 1260: 2,
    1300: 1, 1310: 4, 1400: 6, 1500: 4, 1510: 4, 1520: 4
}

# Initialize MRP DataFrame for each part
weeks = list(range(1, 13))  # Weeks from 1 to 12
mrp_frames = {}

for part, quantity_per_tent in part_quantities.items():
    # Creating a DataFrame to store MRP details for each part
    data = {
        'Week': weeks,
        'Gross Requirements': [x * quantity_per_tent for x in tent_gross_requirements],
        'Projected Available': [0] * 12,  # This will need adjustment
        'Net Requirement': [0] * 12,
        'Planned Order Receipt': [0] * 12,
        'Planned Order Release': [0] * 12
    }
    df_mrp = pd.DataFrame(data)
    df_mrp.set_index('Week', inplace=True)

    # Assuming the projected available for the tent starts at 75 and then goes to 0
    # Adjust the Projected Available and Net Requirements
    projected_available = 75 if part == 1000 else 0  # Starting inventory for the tent component
    for week in weeks:
        if week == 1:
            df_mrp.at[week, 'Projected Available'] = projected_available
        else:
            projected_available = max(0, projected_available - df_mrp.at[week, 'Gross Requirements'])

        df_mrp.at[week, 'Net Requirement'] = max(0, df_mrp.at[week, 'Gross Requirements'] - df_mrp.at[week, 'Projected Available'])
        # Planned Order Receipt matches Net Requirement (assuming immediate fulfillment)
        df_mrp.at[week, 'Planned Order Receipt'] = df_mrp.at[week, 'Net Requirement']
        # Planned Order Release could be a week before requirement, this part is hypothetical
        if week > 1:
            df_mrp.at[week - 1, 'Planned Order Release'] = df_mrp.at[week, 'Planned Order Receipt']

    # Store DataFrame in a dictionary for later use or display
    mrp_frames[part] = df_mrp

# Example: Display the MRP for part 1110
print("MRP for Part 1110 (Nylon Fabric):")
print(mrp_frames[1110])

MRP for Part 1110 (Nylon Fabric):
      Gross Requirements  Projected Available  Net Requirement  \
Week                                                             
1                      0                    0                0   
2                    900                    0              900   
3                    900                    0              900   
4                      0                    0                0   
5                    900                    0              900   
6                      0                    0                0   
7                    900                    0              900   
8                    900                    0              900   
9                    900                    0              900   
10                     0                    0                0   
11                   900                    0              900   
12                   900                    0              900   

      Planned Order Receipt  Planned Orde